本次任务参考了github@edwardfang88

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

# Display and Plotting
import matplotlib.pylab as plt
import seaborn as sns


In [4]:
filename_ts = r'./data.csv'
df=pd.read_csv(filename_ts,encoding='gbk')
print(df.columns)
df.head(5)

Index(['Unnamed: 0', 'custid', 'trade_no', 'bank_card_no',
       'low_volume_percent', 'middle_volume_percent',
       'take_amount_in_later_12_month_highest',
       'trans_amount_increase_rate_lately', 'trans_activity_month',
       'trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'regional_mobility', 'student_feature',
       'repayment_capability', 'is_high_user', 'number_of_trans_from_2011',
       'first_transaction_time', 'historical_trans_amount',
       'historical_trans_day', 'rank_trad_1_month', 'trans_amount_3_month',
       'avg_consume_less_12_valid_month', 'abs',
       'top_trans_count_last_1_month', 'avg_price_last_12_month',
       'avg_price_top_last_12_valid_month', 'reg_preference_for_trad',
       'trans_top_time_last_1_month', 'trans_top_time_last_6_month',
       'consume_top_time_last_1_month', 'consume_top_time_last_6_month',
       'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_mont

,Unnamed: 0,custid,trade_no,bank_card_no,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,5,2791858,20180507115231274000000023057383,卡号1,0.01,0.99,0,0.90,0.55,0.313,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,10,534047,20180507121002192000000023073000,卡号1,0.02,0.94,2000,1.28,1.00,0.458,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,12,2849787,20180507125159718000000023114911,卡号1,0.04,0.96,0,1.00,1.00,0.114,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,13,1809708,20180507121358683000000388283484,卡号1,0.00,0.96,2000,0.13,0.57,0.777,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,14,2499829,20180507115448545000000388205844,卡号1,0.01,0.99,0,0.46,1.00,0.175,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [7]:
#student_feature缺失太多直接丢弃,其他没有业务价值
drop_columns=['bank_card_no','source','id_name','custid','student_feature','trade_no']
df.drop(drop_columns,axis=1,inplace=True)

#日期处理为日期格式datetime，并生成新的一列 loans_time
df['latest_query_time']=pd.to_datetime(df['latest_query_time'])
df['loans_latest_time']=pd.to_datetime(df['loans_latest_time'])

#将timedelta格式转化为float类型(秒)
df['loans_time']=df['loans_latest_time']-df['latest_query_time']
df['loans_time']=df['loans_time'].dt.total_seconds().dropna()

#两列日期格式数据处理直接丢弃
datatime_data=['loans_latest_time','latest_query_time']
df.drop(datatime_data,axis=1,inplace=True)

#将城市进行数值化
map_dict={'一线城市':1,'二线城市':2,'三线城市':3,'境外':4,'其他':0}
df['reg_preference_for_trad']=df['reg_preference_for_trad'].map(map_dict)
df.head()

,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,loans_time
0,5,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,...,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0,-518400.0
1,10,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,...,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0,172800.0
2,12,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,...,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0,-345600.0
3,13,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,...,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0,-172800.0
4,14,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,...,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0,-8467200.0


In [8]:
#缺失值以中位值进行填充
from sklearn.preprocessing import Imputer
columns=list(df)
imp=Imputer(missing_values=np.nan,strategy='median')
df=imp.fit_transform(df)
print('number after processing:',df.shape)
df=pd.DataFrame(df,columns=columns)

number after processing: (4754, 83)


In [9]:
#丢掉方差为0即没有变化的列
df.drop(df.columns[df.std()==0],axis=1,inplace=True)
df.shape

(4754, 83)

In [10]:
train_x=df.drop(['status'],axis=1)
train_y=df['status']

print(train_x.shape)
train_x.head()

(4754, 82)


,Unnamed: 0,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,loans_time
0,5.0,0.01,0.99,0.0,0.90,0.55,0.313,17.0,27.0,26.0,...,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0,-518400.0
1,10.0,0.02,0.94,2000.0,1.28,1.00,0.458,19.0,30.0,14.0,...,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0,172800.0
2,12.0,0.04,0.96,0.0,1.00,1.00,0.114,13.0,68.0,22.0,...,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0,-345600.0
3,13.0,0.00,0.96,2000.0,0.13,0.57,0.777,22.0,14.0,6.0,...,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0,-172800.0
4,14.0,0.01,0.99,0.0,0.46,1.00,0.175,13.0,66.0,42.0,...,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0,-8467200.0


In [11]:
#拆分测试集和训练集
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(train_x,train_y,test_size=0.3,random_state=2018)
print(x_train.shape)
print(x_test.shape)
print(y_test.shape)

(3327, 82)
(1427, 82)
(1427,)


In [12]:
#引入模型进行训练
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


In [13]:
#定义计算iv值的函数
def calc_iv(df, feature, target, pr=False):
    """
    Set pr=True to enable printing of output.

    Output:
      * iv: float,
      * data: pandas.DataFrame
    """
    lst = []
    df[feature] = df[feature].fillna('NULL')

    for i in range(df[feature].nunique()):  # nuinque()是查看该序列(axis=0/1对应着列或行)的不同值的数量个数
        val = list(df[feature].unique())[i]
        lst.append([feature,
                    val,  # Value
                    df[df[feature] == val].count()[feature],  # all
                    df[(df[feature] == val) & (df[target] == 0)].count()[feature],  # good rate
                    df[(df[feature] == val) & (df[target] == 1)].count()[feature]])  # bad rate
    data = pd.DataFrame(lst, columns=['Variable', 'Value', 'All', 'Good', 'Bad'])

    data['Share'] = data['All'] / data['All'].sum()
    data['Bad Rate'] = data['Bad'] / data['All']
    data['Distribution Good'] = (data['All'] - data['Bad']) / (data['All'].sum() - data['Bad'].sum())
    data['Distribution Bad'] = data['Bad'] / data['Bad'].sum()
    data['WoE'] = np.log(data['Distribution Good'] / data['Distribution Bad'])

    data = data.replace({'WoE': {np.inf: 0, -np.inf: 0}})

    data['IV'] = data['WoE'] * (data['Distribution Good'] - data['Distribution Bad'])

    data = data.sort_values(by=['Variable', 'Value'], ascending=[True, True])
    data.index = range(len(data.index))

    if pr:
        print(data)
        print("IV = ", data['IV'].sum())

    iv = data['IV'].sum()

    return iv, data

column_headers = list(df.columns.values)
# print(column_headers)

d=[]
for x in column_headers:
    IV_1, data = calc_iv(df, x, 'status')
    d.append(IV_1)
#整理成Series类型并合并
column_headers=pd.Series(column_headers,name='feature')
d=pd.Series(d,name='iv_value')

# print(column_headers)
iv_result=pd.concat([column_headers,d],axis=1)
iv_result.sort_values(by='iv_value',ascending=False)

#     print('{}： {}'.format(x, IV_1))

,feature,iv_value
52,loans_score,0.615595
30,trans_fail_top_count_enum_last_1_month,0.598137
64,history_fail_fee,0.507044
43,apply_score,0.488201
14,first_transaction_time,0.486122
41,first_transaction_day,0.486122
11,repayment_capability,0.439096
34,max_cumulative_consume_later_1_month,0.395649
56,loans_overdue_count,0.387895
20,abs,0.372721


In [14]:
#筛选IV值大于0.02的作为有效可预测列
iv_below=iv_result[iv_result['iv_value']<0.02]
deleted_cols_by_iv=list(iv_below.feature)
deleted_cols_by_iv.remove('status')

In [15]:
x_train.drop(deleted_cols_by_iv,axis=1,inplace=True)
x_test.drop(deleted_cols_by_iv,axis=1,inplace=True)

In [16]:
# 此时选择将x_train进行标准化
from sklearn.preprocessing import StandardScaler
columns=list(x_train)
x_train=StandardScaler().fit_transform(x_train)
x_train=pd.DataFrame(x_train,columns=columns)

x_test=StandardScaler().fit_transform(x_test)
x_test=pd.DataFrame(x_test,columns=columns)

In [17]:
#定义返回y_pred和y_pred_proba的函数，并能打印训练集和测试集的精度以及ROC曲线面积
def model_metrics(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train,y_train)
    # 预测
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    
    y_train_proba = clf.predict_proba(X_train)[:,1]
    y_test_proba = clf.predict_proba(X_test)[:,1]
    
    # 准确率
    print('[准确率]', end = ' ')
    print('训练集：', '%.4f'%accuracy_score(y_train, y_train_pred), end = ' ')
    print('测试集：', '%.4f'%accuracy_score(y_test, y_test_pred))
    
#     auc取值：用roc_auc_score或auc
    print('[auc值]', end = ' ')
    print('训练集：', '%.4f'%roc_auc_score(y_train, y_train_proba), end = ' ')
    print('测试集：', '%.4f'%roc_auc_score(y_test, y_test_proba))
    
    #加上return方便后续计算混淆矩阵效果
    return y_test_proba,y_test_pred

In [18]:
#调入随机森林模型，无调参之前看下分数
rf=RandomForestClassifier()

model_metrics(rf,x_train,x_test,y_train,y_test)


[准确率] 训练集： 0.9856 测试集： 0.7547
[auc值] 训练集： 0.9992 测试集： 0.7036


(array([0.1, 0.1, 0.6, ..., 0.2, 0.4, 0.3]),
 array([0., 0., 1., ..., 0., 0., 0.]))

In [19]:
feature_importance=sorted(zip(rf.feature_importances_,list(x_train)),reverse=True)
feature_importance,len(feature_importance)


([(0.06865617821315113, 'trans_fail_top_count_enum_last_1_month'),
  (0.058141666252044935, 'history_fail_fee'),
  (0.039682334140718194, 'loans_score'),
  (0.03166905380582934, 'latest_one_month_fail'),
  (0.023671084682482205, 'loans_overdue_count'),
  (0.023590331964550224, 'apply_score'),
  (0.018670673906371272, 'trans_amount_3_month'),
  (0.018393959431901995, 'abs'),
  (0.017909332797579593, 'loans_long_time'),
  (0.017680187571394064, 'trans_days_interval_filter'),
  (0.01683301126779656, 'trans_amount_increase_rate_lately'),
  (0.01598676064793535, 'trans_day_last_12_month'),
  (0.015537528619505716, 'first_transaction_day'),
  (0.015424227352165549, 'consfin_credit_limit'),
  (0.015149695281740438, 'repayment_capability'),
  (0.015066979518995166, 'latest_query_day'),
  (0.015028082176065712, 'consfin_avg_limit'),
  (0.014742632097704397, 'first_transaction_time'),
  (0.014520037617084541, 'pawns_auctions_trusts_consume_last_6_month'),
  (0.014519811157755611, 'trans_activity

In [20]:
#按照排序,使用前30列为有用列,其他为无用列
useless=[]
for feature in x_train.columns:
    if feature in [t[1] for t in feature_importance[30:]]:
        useless.append(feature)
x_train.drop(useless,axis=1,inplace=True)
x_test.drop(useless,axis=1,inplace=True)
x_train.shape

(3327, 30)